In [1]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED

c:\Tools\MiniConda\envs\pyshred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
import os

# Initialize ParametricSHREDDataManager
manager = ParametricSHREDDataManager(
    lags = 20,
    train_size = 0.8,
    val_size = 0.1,
    test_size = 0.1,
    compression = 50,
    )

# Add data to manager (with sensors)
dataset = np.load('kuramoto_sivashinsky\KuramotoSivashinsky_data.npz')
data = dataset['u'] # shape (500, 201, 100)
mu = dataset['mu'] # shape (500, 201, 2)
mobile_sensors = [
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=True),
    forward_backward_walk(start=0, end = data.shape[2], timesteps=data.shape[1], forward_first=False),
]

manager.add(
    data=data,
    random_sensors=2,
    stationary_sensors=[(0,), (1,)],
    # mobile_sensors=mobile_sensors,
    params=mu,
    id = 'KS'
)


hello


In [40]:
#seperate sensors from params, so when user enters new data, it is clear what the sensor order is and what the param order is when there are sensor data and param data from multiple fields
# do multiple fields make sense in parametric shred? 
# can different fields have different params? the result of 'mixing up' params of one field and keeping another field params constant to see effect??
manager.sensor_measurements

,trajectory,KS-0,KS-1,KS-2,KS-3
0,0,0.009270,-0.113056,1.000000,1.093962
1,0,0.032116,-0.116359,0.376289,0.546796
2,0,-0.000694,-0.059762,0.450057,0.619256
3,0,-0.033779,-0.017287,0.511200,0.682191
4,0,-0.061429,0.010436,0.556852,0.728624
...,...,...,...,...,...
100495,499,-0.045187,-0.332669,-0.658349,-0.446285
100496,499,0.002631,-0.069085,-0.602322,-0.400786
100497,499,0.045830,0.135359,-0.181651,-0.137417
100498,499,0.084961,0.291639,0.561369,0.286482


In [41]:
manager.sensor_summary

,field id,sensor id,sensor type,location/trajectory
0,KS,KS-0,stationary (randomly selected),"(50,)"
1,KS,KS-1,stationary (randomly selected),"(41,)"
2,KS,KS-2,stationary (user selected),"(0,)"
3,KS,KS-3,stationary (user selected),"(1,)"


In [42]:
manager.params

,KS param 0,KS param 1
0,1.423285,1.568899
1,1.423285,1.568899
2,1.423285,1.568899
3,1.423285,1.568899
4,1.423285,1.568899
...,...,...
100495,1.160304,1.782302
100496,1.160304,1.782302
100497,1.160304,1.782302
100498,1.160304,1.782302


In [43]:
# Get train/val/test datasets
train_set, val_set, test_set = manager.preprocess()

# # Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor_dataset.X.shape)
print('train Y:', train_set.reconstructor_dataset.Y.shape)
print('val X:', val_set.reconstructor_dataset.X.shape)
print('val Y:', val_set.reconstructor_dataset.Y.shape)
print('test X:', test_set.reconstructor_dataset.X.shape)
print('test Y:', test_set.reconstructor_dataset.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([80400, 21, 6])
train Y: torch.Size([80400, 50])
val X: torch.Size([10050, 21, 6])
val Y: torch.Size([10050, 50])
test X: torch.Size([10050, 21, 6])
test Y: torch.Size([10050, 50])


In [44]:
result = manager.postprocess(train_set.reconstructor_dataset.Y)
print(result['KS'].shape)

(400, 201, 100)


In [45]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_set, val_set, num_epochs=100, patience=100, verbose=True)


Fitting Reconstructor...


Epoch 1/100:   8%|▊         | 104/1257 [00:01<00:10, 111.75batch/s, loss=0.0198, L2=0.262]

KeyboardInterrupt: 

Epoch 1/100:   8%|▊         | 105/1257 [00:20<00:10, 111.75batch/s, loss=0.0198, L2=0.262]

In [ ]:
reconstruction = shred.reconstruct(test_set.reconstructor_dataset.X).detach().cpu().numpy()
reconstruction.shape

In [ ]:
### given sensor measurements of shape (ntraj, ntimes, nsensors) -> (ntraj*ntimes, lags, nsensors) and params of (ntraj, ntimes, nparams) -> (ntaj*ntimes, lags, nsensors)


In [ ]:
# development testing
train_set.reconstructor.X[201,:,0]

In [ ]:
# NEW DATA
# n_traj, n_times, nsensors
# n_traj, n_times, nparams


